# Open Evaluation - Consulta Abierta en Lenguaje Natural

Este notebook permite realizar consultas abiertas en lenguaje natural y obtener una respuesta JSON generada por el modelo entrenado.


## Librerías


In [ ]:
import os
import json
import random

import torch
import shared_functions as custom_sharfun

from peft import PeftModel
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from transformers import logging as hf_logging
hf_logging.set_verbosity_warning()


## Configuración


In [ ]:
# IMPORTANTE: dejar las mismas configuraciones que train.ipynb
# ------------------------------------------------------------------

# Data / tokenization
MAX_LENGTH = 1252  # Mismo valor que en train.ipynb

# Evaluación
GEN_MAX_NEW_TOKENS = 377  # Mismo valor que en train.ipynb

# Configuración del dispositivo
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

GLB_SEED = 42
torch.manual_seed(GLB_SEED)
random.seed(GLB_SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(GLB_SEED)


## Cargar Modelo Entrenado


In [ ]:
MODEL_NAME = "Qwen/Qwen3-0.6B-Base"
WEIGHTS_PATH = "weights.pt"  # Pesos en la raíz del proyecto
OUTPUT_DIR = "output/results/v01"
ADAPTER_DIR = os.path.join(OUTPUT_DIR, "modfinal")

print("Cargando tokenizer y modelo...")

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR, trust_remote_code=True)

# Configuración de cuantización
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Cargar modelo base
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Cargar adaptadores LoRA
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()

print("✅ Modelo cargado exitosamente")


## Función de Consulta Abierta


In [ ]:
def generar_orden_compra(texto_natural: str, verbose: bool = True) -> dict:
    """
    Genera una orden de compra en formato JSON a partir de texto en lenguaje natural.
    
    Args:
        texto_natural: Texto en lenguaje natural describiendo la orden de compra
        verbose: Si True, imprime información adicional del proceso
    
    Returns:
        dict: JSON con la orden de compra estructurada
    """
    if verbose:
        print("\n" + "="*80)
        print("📝 TEXTO DE ENTRADA:")
        print("="*80)
        print(texto_natural)
        print("\n🔄 Generando JSON...\n")
    
    # Generar el JSON usando el modelo
    raw_output = custom_sharfun.generate_json_raw(
        text=texto_natural,
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        max_length=MAX_LENGTH,
        tokenizer=tokenizer,
        model=model,
        device=DEVICE
    )
    
    # Extraer JSON del texto generado
    json_obj = custom_sharfun.extract_json_from_text(raw_output)
    
    if json_obj is None:
        if verbose:
            print("⚠️  No se pudo extraer un JSON válido")
            print("\n📄 Salida cruda del modelo:")
            print(raw_output)
        return {}
    
    if verbose:
        print("="*80)
        print("✅ JSON GENERADO:")
        print("="*80)
        print(json.dumps(json_obj, indent=2, ensure_ascii=False))
        print("="*80)
    
    return json_obj


## Ejemplos de Uso

### Ejemplo 1: Pedido Simple


In [ ]:
texto_ejemplo_1 = """
Hola, necesito hacer un pedido urgente.

Mi nombre es María García y mi correo es maria.garcia@example.com
Mi teléfono es +34-555-123-456

Quiero comprar:
- 2 teclados inalámbricos
- 1 mouse ergonómico

Mi dirección es: Calle Mayor 123, Madrid, España, código postal 28013

Necesito envío express para antes del 15 de diciembre de 2025.

Gracias!
"""

resultado_1 = generar_orden_compra(texto_ejemplo_1)


### Ejemplo 2: Pedido con Información Incompleta


In [ ]:
texto_ejemplo_2 = """
Hola, quiero ordenar 5 laptops para mi oficina.
Puedes contactarme al correo: juan.perez@company.com
Prefiero recogerlo yo mismo en la tienda.
"""

resultado_2 = generar_orden_compra(texto_ejemplo_2)


### Ejemplo 3: Tu Propia Consulta

**Escribe aquí tu propio texto para generar una orden de compra:**


In [ ]:
# ✏️ Modifica el texto a continuación con tu propia orden de compra
mi_texto = """
Escribe aquí tu orden de compra en lenguaje natural...
"""

mi_resultado = generar_orden_compra(mi_texto)


## Consulta Interactiva

Usa esta celda para probar múltiples consultas de forma rápida:


In [ ]:
# Escribe tu consulta aquí y ejecuta la celda
consulta = input("Escribe tu orden de compra en lenguaje natural:\n")
resultado = generar_orden_compra(consulta)


## Guardar Resultado

Si quieres guardar algún resultado específico:


In [ ]:
# Guardar el último resultado generado
output_file = "mi_orden_compra.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"✅ Resultado guardado en: {output_file}")
